# Análisis de datos de nosepoke

## 0. Importación de librerías

In [25]:
import pandas as pd
import os
import docx2txt
from datetime import datetime, timedelta
import re

## 0.1.Funciones auxiliares

In [2]:
path="D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE/"
pathMachos=f"{path}/MACHOS/"
pathHembras=f"{path}/HEMBRAS/"

In [26]:
def separarLinea(linea:str):
  """
  Cada línea de los archivos tiene una estructura "HH:MM:SS.MS -> DATOS".
  En caso de que no haya una entrada de tiempo se pone como hora '-1'
  
  Parameters:
  linea (String): linea de texto extraída del archivo de datos
  etiqueta (String): indica si hay más de un registro en el archivo "SPLIT" o si no tiene el tiempo "TIEMPO"

  Returns:
  hora, registro (String, String): hora y datos por separado.
  """
  if len(linea.split('->'))==2:
    hora=linea.split('->')[0]
    registro=linea.split('->')[1]
  else:
    hora=-1
    registro=linea
  return hora, registro

In [27]:
def leerTexto(path):
  text=docx2txt.process(path)
  if len(text)==0:
        print(f'Archivo vacío: {path}')
  # Los archivos donde los datos están explícitos no tienen '--------------------------------------------------'
  esCSV=text.find('--------------------------------------------------') < 0
  text=text.split('\n')
  # Se eliminan líneas vacías
  text= list(filter(None, text))
  return text, esCSV

In [28]:
def quitarEncabezadoNoCSV(textArray):
  for index,line in enumerate(textArray):
    if line.find('Van a empezar los 50 ensayos') != -1 or line.find('Ensayo numero') != -1:
      return textArray[index:]
  return []

In [29]:
def extraerValorNumerico(texto):
  return float(re.findall(r'\d+\.?\d*', texto)[0])

In [30]:
def promedioInverso(promedioActual, promedioAnterior, n):
  if n == 1:
    return promedioActual
  return n*promedioActual - promedioAnterior*(n-1)

In [31]:
def procesarEnsayo(ensayoActual, esCSV, ultimoEnsayo, tipoPrueba):
  if tipoPrueba == 'HAB':
    if esCSV:
      aciertos=extraerValorNumerico(ensayoActual[1])
      tiempoTotalImpulsivas=extraerValorNumerico(ensayoActual[3])
      omisiones=extraerValorNumerico(ensayoActual[4])
      categorias=extraerValorNumerico(ensayoActual[6])
      correctasDer=extraerValorNumerico(ensayoActual[7])
      latenciaPromedio=extraerValorNumerico(ensayoActual[9])
      maxAciertosSeguidos=extraerValorNumerico(ensayoActual[10])
    else:
      aciertos=extraerValorNumerico(ensayoActual[0])
      tiempoTotalImpulsivas=extraerValorNumerico(ensayoActual[2])
      omisiones=extraerValorNumerico(ensayoActual[3])
      categorias=extraerValorNumerico(ensayoActual[5])
      correctasDer=extraerValorNumerico(ensayoActual[6])
      latenciaPromedio=extraerValorNumerico(ensayoActual[8])
      maxAciertosSeguidos=extraerValorNumerico(ensayoActual[9])

    n = aciertos+omisiones
    esAcierto = aciertos != ultimoEnsayo['Aciertos acomulados']

    nuevoEnsayo = {
        "No. ensayo": n,
        "Fue acierto": esAcierto,
        "Fue acierto derecho": correctasDer != ultimoEnsayo['Total correctas derecha'] if esAcierto else '',
        "%Aciertos": aciertos/n,
        "%Omisiones":omisiones/n,
        "Latencia de respuesta": promedioInverso(latenciaPromedio, ultimoEnsayo['Latencia promedio'], aciertos) if esAcierto else 0,
        "Tiempo en respuestas impulsivas": tiempoTotalImpulsivas - ultimoEnsayo['Tiempo total impulsivas'],
        "Categorías":categorias,
        "Máximo aciertos seguidos":maxAciertosSeguidos,
        "Aciertos acomulados":aciertos,
        "Tiempo total impulsivas":tiempoTotalImpulsivas,
        "Total correctas derecha":correctasDer,
        "Latencia promedio":latenciaPromedio
        }
  return nuevoEnsayo

In [36]:
def extraerDatosBloque(texto, esCSV:bool, fileName, diaEnsayo, bloque, idAnimal, tipoPrueba, etiqueta):
    """
    Se carga el registro de cada animal en formato de texto y se realiza la 
    extracción de datos
    
    Parameters:
    texto (Array of String): Arreglo donde cada entrada es una línea del texto
    fileName (String): dirección del archivo de registro
    diaEnsayo (Int): día del ensyo en que se tomaron los datos
    bloque (Int): bloque dentro del día en que se tomaron los datos
    tipoPrueba (String): Cadena de texto con el nombre de la prueba
    etiqueta (String): Indica si hay más de un registro en el archivo "SPLIT" o si no tiene el tiempo "TIEMPO"

    Returns:
    dataDict (Pandas Dataframe): diccionario con los datos del ensayo
    """

    dataDict={
        "Dirección": [],
        "Día": [],
        "Bloque": [],
        "No. ensayo": [],
        "Fue acierto":[],
        "Fue acierto derecho":[],
        "%Aciertos":[],
        "%Omisiones":[],
        "Latencia de respuesta (ms)":[],
        "Tiempo en respuestas impulsivas (s)":[],
        "Categorías":[],
        "Máximo aciertos seguidos":[],
        "Tiempo total del bloque (s)":[],
        }

    ultimoEnsayo={
        "No. ensayo": -1,
        "Fue acierto":-1,
        "Fue acierto derecho":-1,
        "%Aciertos":-1,
        "%Omisiones":-1,
        "Latencia de respuesta":-1,
        "Tiempo en respuestas impulsivas":-1,
        "Categorías":-1,
        "Máximo aciertos seguidos":-1,
        "Aciertos acomulados":0,
        "Tiempo total impulsivas":0,
        "Total correctas derecha":0,
        "Total correctas izquierda":0,
        "Latencia promedio":0
        }
    # Variables para registrar el tiempo
    hayTimestamps = etiqueta != "TIEMPO"
    horaInicio=""
    horaFin=""
    
    ensayoActual = []
    
    if esCSV:
      for linea in texto:
        # Cada línea se espera que tenga la siguiente estructura "12:20:12.886 -> Transmitiendo"
        hora, registro = separarLinea(linea)
        
        # Registra la primera hora del documento y luego, la reemplaza por el inicio de los ensayos, si lo hay
        if hayTimestamps:
          if hora == -1:
            print(f'El archivo {fileName} tiene una línea sin marca de tiempo')
          else:
            if registro.find("Van a empezar los 50 ensayos")>-1 or horaInicio=="":
              horaInicio=datetime.strptime(hora.split('.')[0],"%H:%M:%S")
            horaFin=datetime.strptime(hora.split('.')[0],"%H:%M:%S")
            if registro.find("Terminan bloque de 50 ensayos")>-1:
              break
        ensayoActual = registro.split(',')
        if len(ensayoActual) == 11 and registro.find('Ensayo')==-1:
          ultimoEnsayo = procesarEnsayo(ensayoActual, esCSV, ultimoEnsayo, tipoPrueba)
          dataDict["Dirección"].append(fileName)
          dataDict["Día"].append(diaEnsayo)
          dataDict["Bloque"].append(bloque)
          dataDict["No. ensayo"].append(ultimoEnsayo["No. ensayo"])
          dataDict["Fue acierto"].append(ultimoEnsayo["Fue acierto"])
          dataDict["Fue acierto derecho"].append(ultimoEnsayo["Fue acierto derecho"])
          dataDict["%Aciertos"].append(ultimoEnsayo["%Aciertos"])
          dataDict["%Omisiones"].append(ultimoEnsayo["%Omisiones"])
          dataDict["Latencia de respuesta (ms)"].append(ultimoEnsayo["Latencia de respuesta"])
          dataDict["Tiempo en respuestas impulsivas (s)"].append(ultimoEnsayo["Tiempo en respuestas impulsivas"])
          dataDict["Categorías"].append(ultimoEnsayo["Categorías"])
          dataDict["Máximo aciertos seguidos"].append(ultimoEnsayo["Máximo aciertos seguidos"])
    else:
      # Se eliminan las líneas iniciales
      texto = quitarEncabezadoNoCSV(texto)
      
      # Se lee el archivo línea a línea
      logging=False
      for linea in texto:
        # Cada línea se espera que tenga la siguiente estructura "12:20:12.886 -> Transmitiendo"
        hora, registro = separarLinea(linea)
        
        # Registra la primera hora del documento y luego, la reemplaza por el inicio de los ensayos, si lo hay
        if hayTimestamps:
          if registro.find("Van a empezar los 50 ensayos")>-1 or horaInicio=="":
            horaInicio=datetime.strptime(hora.split('.')[0],"%H:%M:%S")
          horaFin=datetime.strptime(hora.split('.')[0],"%H:%M:%S")
          if registro.find("Terminan bloque de 50 ensayos")>-1:
            break
            
        # Si inicia un ensayo, se extraerá el bloque hasta que termine para procesarlo
        if registro.find('Aciertos:')>-1:
          logging=True
        elif logging and registro.find('--------------------------------------------------')>-1:
          logging=False
          ultimoEnsayo = procesarEnsayo(ensayoActual, esCSV, ultimoEnsayo, tipoPrueba)
          ensayoActual = []
          dataDict["Dirección"].append(fileName)
          dataDict["Día"].append(diaEnsayo)
          dataDict["Bloque"].append(bloque)
          dataDict["No. ensayo"].append(ultimoEnsayo["No. ensayo"])
          dataDict["Fue acierto"].append(ultimoEnsayo["Fue acierto"])
          dataDict["Fue acierto derecho"].append(ultimoEnsayo["Fue acierto derecho"])
          dataDict["%Aciertos"].append(ultimoEnsayo["%Aciertos"])
          dataDict["%Omisiones"].append(ultimoEnsayo["%Omisiones"])
          dataDict["Latencia de respuesta (ms)"].append(ultimoEnsayo["Latencia de respuesta"])
          dataDict["Tiempo en respuestas impulsivas (s)"].append(ultimoEnsayo["Tiempo en respuestas impulsivas"])
          dataDict["Categorías"].append(ultimoEnsayo["Categorías"])
          dataDict["Máximo aciertos seguidos"].append(ultimoEnsayo["Máximo aciertos seguidos"])

        if logging:
          ensayoActual.append(registro)

    try:
      segundosTotales=(horaFin-horaInicio).total_seconds()
      if segundosTotales < 0:
          horaFin -= timedelta(hours=1)
          horaInicio -= timedelta(hours=1)
          segundosTotales=(horaFin-horaInicio).total_seconds()
      dataDict["Tiempo total del bloque (s)"] = [segundosTotales]*len(dataDict['Dirección'])
    except:
      dataDict["Tiempo total del bloque (s)"] = [-1]*len(dataDict['Dirección'])
      print(fileName +", No se puede calcular el tiempo total del experimento\n")
          
    if ultimoEnsayo["No. ensayo"] != len(dataDict['Dirección']):
      print(f'El bloque {fileName} tiene {len(dataDict["Dirección"])} ensayos, pero termina en {ultimoEnsayo["No. ensayo"]}' )
    return pd.DataFrame.from_dict(dataDict)

## 1. Extraer lista de los registros

Lo primero que se hará es extraer la lista de los archivos que hay disponibles. Para esto, se recorrerán de forma iterativa las carpetas donde están los datos almacenados

In [3]:
def listaDeRegistros(base):
    """
    Toma una ubicación y realiza un proceso iterativo buscando en todos los 
    archivos que se encuentren almacenados
    
    Parameters:
    base (str): dirección del directorio base donde se buscarán los archivos que
    haya presentes

    Returns:
    registros (Array of str): lista de registros presentes en la carpeta

   """
    registros=[]
    for i in os.listdir(base):
        if os.path.isdir(base+i):
            registros+=listaDeRegistros(base+i+"/")
        elif os.path.isfile(base+i):
            registros.append(base+i)
    return registros

In [4]:
def listaDeAnimales(registros, sexChar):
    """
    Toma el nombre de archivo de un registro y la divide, extrayendo los datos
    correspondientes, asumiendo que cumple con la convención de formato
    [DD].[MM].[AAAA].[Sex]_C[#Caja]_[#Animal]_[IdPrueba]_D[#dia]_B[#Bloque].[optional].docx
    
    Parameters:
    registros (Array of str): lista de registros
    sexo (Char): Caracter que puede ser 'H' para hembras y 'M' para machos

    Returns:
    data (Pandas dataframe): dataframe de pandas con los datos de cada registro
   """
   
    dia=[]
    mes=[]
    año=[]
    sexo=[]
    caja=[]
    marca=[]
    tipoPrueba=[]
    diaEnsayo=[]
    bloque=[]
    nombreArchivo=[]
    etiqueta=[]
    
    for registro in registros:
        try:
            # Ejemplo de registro '3.12.2021.H_C21_5_REV_AZ_D8_B1.docx'
            direccion=registro.split("/")[-1:][0].split(".")
            
            # '3' '12' '2021'
            dia.append(int(direccion[0]))
            mes.append(int(direccion[1]))
            año.append(int(direccion[2]))
            
            # 'H_C21_5_REV_AZ_D8_B1'
            metadata=[]
            for i in direccion[3].split('_'):
                i = i.strip()
                if ' ' in i.strip():
                    for j in i.split(' '):
                        if j!='':
                            metadata.append(j)
                else:
                    metadata.append(i)
                        
            
            if sexChar in metadata[0]:
                sexo.append(metadata.pop(0).strip())
            else:
                sexo.append(sexChar)
                # print("Error al leer el sexo del animal: "+registro)
                
            if 'C' in metadata[0]:
                caja.append(metadata.pop(0).strip()[1:])
            else:
                caja.append(-1)
                print("Error al leer el identificador de la caja: "+registro)
                
            try:
                int(metadata[0])
                marca.append(metadata.pop(0).strip())
            except:
                marca.append(-1)
                print("Error al leer el numero del animal: "+registro)
                
            if 'B' in metadata[-1]:
                bloque.append(metadata.pop().strip()[1:])
            else:
                bloque.append(-1)
                print("Error al leer el bloque: "+registro)
                
            if 'D' in metadata[-1]:
                diaEnsayo.append(metadata.pop().strip()[1:])
            else:
                diaEnsayo.append(-1)
                print("Error al leer el bloque: "+registro)
            
            tipo=""
            for i in metadata:
                tipo+=(i.strip()+'_')
            tipoPrueba.append(tipo.strip()[:-1])
            
            if len(direccion)==6:
                etiqueta.append(direccion[4].strip())
            elif re.search('ESTANDARI', registro, re.IGNORECASE):
                etiqueta.append('estandarizacion')
            else:
                etiqueta.append('')
            
            nombreArchivo.append(registro)
            
        except:
            print("Error adicional: "+registro)
    d={'dia':dia, 'mes':mes, 'año':año, 'sexo':sexo, 'caja':caja,
       'marca':marca, 'tipoPrueba':tipoPrueba, 'diaEnsayo':diaEnsayo,
       'bloque':bloque, "nombreArchivo":nombreArchivo, 'etiqueta':etiqueta}
    data=pd.DataFrame(data=d)
    return data

In [5]:
def estandarizarTerminosTipoDePrueba(data):
    """
    Se tienen variaciones del identificador usado para los 7 tipos de prueba
    que se realizaron. Este método unifica los diferentes términos usados
    
    Parameters:
    data (Pandas dataframe): dataframe de pandas con los datos de cada registro

    Returns:
    cleanData (Pandas dataframe): dataframe de pandas con los términos unificados para la columna tipoPrueba
    """
    cleanData=data.copy()
    replaceDict={"_HAB":"HAB",
                 "P_DER":"SM_ADQ_DER", "SM_P_DER":"SM_ADQ_DER",
                 "SM_PRE_DER":"SM_ADQ_DER", "SM_ADQ_DER_":"SM_ADQ_DER","SM_DER_ADQ":"SM_ADQ_DER",
                 "SMM_ADQ_DER":"SM_ADQ_DER",
                 "P_IZQ":"SM_ADQ_IZQ","SM_P_IZQ":"SM_ADQ_IZQ","SM_IZQ_ADQ":"SM_ADQ_IZQ",
                 "SM_IZQ":"SM_ADQ_IZQ","SM_PRE_IZQ":"SM_ADQ_IZQ",
                 
                 "R_DER":"SM_REV_DER","SM_R_DER":"SM_REV_DER",
                 "R_IZQ":"SM_REV_IZQ","SM_R_IZQ":"SM_REV_IZQ",
                 
                 "DIS":"RA_ADQ", "DIS_AZ":"RA_ADQ", "ADQ_AZ":"RA_ADQ",
                 
                 "REV_AZ":"RA_REV", "RA":"RA_REV", "REV":"RA_REV", "R_AZ":"RA_REV",
                 }
    cleanData['tipoPrueba'].replace(to_replace=replaceDict, inplace=True)
    return cleanData

In [6]:
registrosHembras=estandarizarTerminosTipoDePrueba(listaDeAnimales(listaDeRegistros(pathHembras), 'H'))

Error adicional: D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE//HEMBRAS/ACTUALIZADA. REGISTRO APRENDIZAJE EXPERIMENTO -iMac de Diego.xlsx
Error adicional: D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE//HEMBRAS/Cronograma Actividades Experimento.xlsx


In [7]:
registrosHembras.head()

,dia,mes,año,sexo,caja,marca,tipoPrueba,diaEnsayo,bloque,nombreArchivo,etiqueta
0,27,8,2021,H,51,1,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,
1,27,8,2021,H,51,1,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,
2,27,8,2021,H,51,2,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,
3,27,8,2021,H,51,2,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,
4,27,8,2021,H,51,3,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,


In [8]:
registrosMachos=estandarizarTerminosTipoDePrueba(listaDeAnimales(listaDeRegistros(pathMachos), 'M'))

In [9]:
registrosMachos.head()

,dia,mes,año,sexo,caja,marca,tipoPrueba,diaEnsayo,bloque,nombreArchivo,etiqueta
0,13,8,2021,M,10,1,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,
1,13,8,2021,M,10,1,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,
2,13,8,2021,M,10,2,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,
3,13,8,2021,M,10,2,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,
4,13,8,2021,M,12,1,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,MD


In [10]:
registrosUnificados=pd.concat([registrosMachos, registrosHembras], ignore_index=True)
registrosUnificados.insert(loc=0, column='IdAnimal', value=registrosUnificados.apply(lambda fila: f"{fila.sexo}-{fila.caja}-{fila.marca}", axis=1))
registrosUnificados.to_excel("registrosMachosYHembras.xlsx")

In [11]:
registrosUnificados.head()

,IdAnimal,dia,mes,año,sexo,caja,marca,tipoPrueba,diaEnsayo,bloque,nombreArchivo,etiqueta
0,M-10-1,13,8,2021,M,10,1,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,
1,M-10-1,13,8,2021,M,10,1,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,
2,M-10-2,13,8,2021,M,10,2,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,
3,M-10-2,13,8,2021,M,10,2,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,
4,M-12-1,13,8,2021,M,12,1,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,MD


### 1.1. Análisis de calidad de los registros

Revisaremos los valores de cada columna para identificar la calidad de los datos

In [12]:
for column in registrosUnificados:
    if column != 'nombreArchivo':
        print({column:registrosUnificados[column].unique()})

{'IdAnimal': array(['M-10-1', 'M-10-2', 'M-12-1', 'M-12-2', 'M-11-1', 'M-11-2',
       'M-11-3', 'M-11-4', 'M-5B6-1', 'M-5B6-4', 'M-5B6-5', 'M-14-1',
       'M-14-2', 'M-21-1', 'M-21-2', 'M-21-3', 'M-24-3', 'M-24-4',
       'M-25-1', 'M-25-3', 'M-25-4', 'M-27-3', 'M-27-4', 'M-31-3',
       'M-31-4', 'M-22-3', 'M-22-4', 'M-23-1', 'M-23-2', 'M-26-1',
       'M-26-2', 'M-28-1', 'M-28-5', 'M-33-4', 'M-33-5', 'M-34-1',
       'M-34-6', 'M-29-5', 'M-29-6', 'M-29-7', 'M-32-1', 'M-32-2',
       'M-32-3', 'M-30-1', 'M-30-2', 'M-30-3', 'M-35-2', 'M-35-4',
       'M-17-1', 'M-19-1', 'M-19-3', 'M-20-2', 'M-20-3', 'M-13-1',
       'M-13-2', 'M-15-1', 'M-15-2', 'M-16-1', 'M-16-2', 'M-17-4',
       'M-17-2', 'H-51-1', 'H-51-2', 'H-51-3', 'H-55-1', 'H-55-2',
       'H-55-3', 'H-56-4', 'H-56-5', 'H-56-6', 'H-56-7', 'H-68-1',
       'H-68-2', 'H-68-3', 'H-60-1', 'H-60-2', 'H-60-3', 'H-61-4',
       'H-61-5', 'H-61-6', 'H-70-1', 'H-70-2', 'H-70-3', 'H-71-4',
       'H-71-6', 'H-63-1', 'H-63-2', 'H-64-3',

In [13]:
for column in registrosMachos:
    if column != 'nombreArchivo':
        print({column:registrosMachos[column].unique()})

{'dia': array([13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
       31,  1, 10, 11, 12, 18,  2,  3,  4,  5,  6,  7,  8,  9],
      dtype=int64)}
{'mes': array([ 8, 12,  1,  2, 11, 10,  9], dtype=int64)}
{'año': array([2021, 2022], dtype=int64)}
{'sexo': array(['M'], dtype=object)}
{'caja': array(['10', '12', '11', '5B6', '14', '21', '24', '25', '27', '31', '22',
       '23', '26', '28', '33', '34', '29', '32', '30', '35', '17', '19',
       '20', '13', '15', '16'], dtype=object)}
{'marca': array(['1', '2', '3', '4', '5', '6', '7'], dtype=object)}
{'tipoPrueba': array(['HAB', 'SM_ADQ_DER', 'SM_ADQ_IZQ', 'SM_REV_DER', 'SM_REV_IZQ',
       'RA_ADQ', 'RA_REV'], dtype=object)}
{'diaEnsayo': array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '17', '11',
       '19', '12', '13', '14', '15', '16', '18', '20', '21', '22', '23',
       '24'], dtype=object)}
{'bloque': array(['1', '2'], dtype=object)}
{'etiqueta': array(['', 'MD', 'TIEMPO', 'estandarizacion', 'estanta

In [14]:
registrosUnificados.query('etiqueta == "estandarizacion"')

,IdAnimal,dia,mes,año,sexo,caja,marca,tipoPrueba,diaEnsayo,bloque,nombreArchivo,etiqueta
176,M-5B6-1,25,8,2021,M,5B6,1,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,estandarizacion
177,M-5B6-1,25,8,2021,M,5B6,1,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,estandarizacion
178,M-5B6-4,25,8,2021,M,5B6,4,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,estandarizacion
179,M-5B6-4,25,8,2021,M,5B6,4,SM_ADQ_IZQ,1,1,D:/Users/USER/OneDrive - Universidad de los An...,estandarizacion
180,M-5B6-5,25,8,2021,M,5B6,5,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,estandarizacion
...,...,...,...,...,...,...,...,...,...,...,...,...
2676,M-5B6-5,25,9,2021,M,5B6,5,RA_REV,10,2,D:/Users/USER/OneDrive - Universidad de los An...,estandarizacion
2681,M-5B6-5,26,9,2021,M,5B6,5,RA_REV,11,1,D:/Users/USER/OneDrive - Universidad de los An...,estandarizacion
2682,M-5B6-5,26,9,2021,M,5B6,5,RA_REV,11,2,D:/Users/USER/OneDrive - Universidad de los An...,estandarizacion
2687,M-5B6-5,28,9,2021,M,5B6,5,RA_REV,12,1,D:/Users/USER/OneDrive - Universidad de los An...,estandarizacion


In [15]:
registrosUnificados.query('etiqueta == "estandarizacion"')['IdAnimal'].unique()

array(['M-5B6-1', 'M-5B6-4', 'M-5B6-5', 'M-14-1'], dtype=object)

In [16]:
registrosUnificados.query('etiqueta == "" & IdAnimal == "M-5B6-4"')

,IdAnimal,dia,mes,año,sexo,caja,marca,tipoPrueba,diaEnsayo,bloque,nombreArchivo,etiqueta


In [ ]:
# Los archivos del 25 y 26 de agosto no están dentro de los archivos
registrosUnificados.query('IdAnimal == "M-22-3" & tipoPrueba == "RA_REV"').head(70)

In [18]:
registrosUnificados.query('caja == "14" & sexo=="M"')

,IdAnimal,dia,mes,año,sexo,caja,marca,tipoPrueba,diaEnsayo,bloque,nombreArchivo,etiqueta
288,M-14-1,31,8,2021,M,14,1,RA_REV,3,1,D:/Users/USER/OneDrive - Universidad de los An...,estandarizacion
289,M-14-2,31,8,2021,M,14,2,RA_REV,7,1,D:/Users/USER/OneDrive - Universidad de los An...,estantadarizacion


Los datos de la columna 'etiqueta' indican errores que se encontraron luego de una revisión manual

| Etiqueta | Descripción |
| --- | --- |
| MD | Datos perdidos, están en blanco |
| TIME/TIEMPO | No tienen el registro del tiempo |
| .resumen_final | Solo arrojaron el resumen de todo el bloque (ensayo 50) |
| SPLIT/Split/split | Archivos que tienen entre 30 y 50 ensayos, los cuales no son consecutivos |
| EXCLUDE |  Machos C35_2 Y C21_2. Uno de ellos no logró el criterio de habituación en 20 días y debimos sacarlo del estudio, mientras que el otro se murió por el camino. De ese animal solo tenemos los datos de habituación que pueden servir (C35_2) |
| error de asignación | Se excluyen dado que se hizo otro protocolo al que correspondía ese día |
| estandarizacion | Se exlcuyen del análisis |

In [19]:
replaceDict={"Split":"SPLIT",
             "split":"SPLIT",
             "TIME":"TIEMPO",
             "estantadarizacion":"estandarizacion"
                 }
registrosUnificados['etiqueta'].replace(to_replace=replaceDict, inplace=True)

In [20]:
# registrosUnificados.loc[registrosUnificados['etiqueta'] == 'SPLIT']['nombreArchivo'].values
registrosUnificados.loc[registrosUnificados['etiqueta'] == 'SPLIT'].head(5)

,IdAnimal,dia,mes,año,sexo,caja,marca,tipoPrueba,diaEnsayo,bloque,nombreArchivo,etiqueta
330,M-23-1,10,12,2021,M,23,1,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,SPLIT
346,M-27-4,11,12,2021,M,27,4,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,SPLIT
383,M-23-2,12,12,2021,M,23,2,HAB,3,1,D:/Users/USER/OneDrive - Universidad de los An...,SPLIT
396,M-27-3,13,12,2021,M,27,3,RA_ADQ,1,2,D:/Users/USER/OneDrive - Universidad de los An...,SPLIT
397,M-27-3,13,12,2021,M,27,3,HAB,4,1,D:/Users/USER/OneDrive - Universidad de los An...,SPLIT


### 1.2. Estadísticos generales
Se mirará el número de datos para cada tipo de prueba en machos y hembras

In [21]:
# Número de archivos por cada tipo de prueba en machos
registrosMachos['tipoPrueba'].value_counts()

RA_REV        1365
HAB            456
RA_ADQ         331
SM_REV_DER     205
SM_ADQ_IZQ     139
SM_REV_IZQ     117
SM_ADQ_DER      84
Name: tipoPrueba, dtype: int64

In [22]:
idMachos = registrosUnificados.query('sexo == "M"')['IdAnimal'].unique()

In [23]:
filesPerAnimal = {}
for id in idMachos:
    fileCount = {}
    countByType = registrosUnificados.query(f'IdAnimal == "{id}" & (etiqueta == "" | etiqueta == "TIEMPO" | etiqueta == "SPLIT")')['tipoPrueba'].value_counts().rename_axis('unique_values').reset_index(name='counts')
    for fileType in countByType['unique_values'].unique():
        fileCount[fileType] = countByType['counts'][countByType['unique_values'] == fileType ].values[0]
    filesPerAnimal[id] = fileCount

In [24]:
pd.DataFrame.from_dict(filesPerAnimal, orient='index').to_excel('Número de archivos por tipo de prueba machos.xlsx')

## 2. Extraer datos de cada tipo de prueba
Vamos a extraer los datos de cada tipo de prueba. Para esto definiremos el enabezado de cada tipo de prueba que vamos a revisar y leeremos cada archivo.

¡OJO! El máximo número de bloques a analizar es 14 por animal

In [14]:
medidas={
    "HAB":{
        "TituloColumna":["Ensayo_", "Aciertos_",
                         "PorcentajeAciertos_","TiempoRespuestasImpulsivas_",
                         "Omisiones_", "PorcentajeOmisiones_", "Categorias_",
                         "CorrectasDerecha_","CorrectasIzquierda_","LatenciaPromedio_",
                         "MaximoAciertosSeguidos_"],
        # 0: Numero, 1: porcentaje, 2: tiempo con "ms", -1: número de ensayo
        "TipoDeDato":[-1, 0, 1, 0, 0, 1, 0, 0, 0, 2, 0],
        "EncabezadoStr":"Ensayo,Aciertos,Porcentaje aciertos,Tiempo total en Respuestas impulsivas,Omisiones,Porcentaje omisiones,Categorias,Total correctas a la derecha,Total correctas a la izquierda,Latencia promedio,Maximo de aciertos seguidos"
    },
}
    

### 2.1. Extraer datos habituación
Cada prueba de habituación cuenta con 50 ensayos. En cada uno de ellos se tienen los siguientes datos:

| Nombre columna | Descripción | Rango |
| --- | --- | --- |
| Ensayo | Número de ensayo | 1 - 50 |
| Aciertos | Número acomulado de aciertos | 0 - 50 |
| Porcentaje aciertos | | 0% - 100% |
| Tiempo total en respuestas impulsivas | Tiempo acomulado que pasa metiendo a la nariz entre ensayos | Continua (segudos) |
| Omisiones | Número acomulado de omisiones | 0 - 50 |
| Porcentaje omisiones | | 0% - 100% |
| Categorias | Cada vez que complete 3 aciertos seguidos se cuenta una categoría| 0 - 16 |
| Total correctas a la derecha | | 0 - 50 |
| Total correctas a la izquierda | | 0 - 50 |
| Latencia promedio | Tiempo promedio en ensayos con aciertos | Contínua (milisegundos) |
| Maximo de aciertos seguidos | | 0- 50 |

Algunos archivos codifican las respuestas impulsivas contándolas y no como el tiempo. Dada la sensibilidad del sensor, esto lleva en algunos casos a números muy elevados.

Se tomará un archivo de habituación y se realizará la extracción de datos. Luego se aplicará a todos los archivos y se revisarán los casos donde no pudo extraer los datos.

Se extraerán los siguientes datos de cada archivo

|dia|bloque|Ensayo|AciertoOmision|LadoAcierto|PorcentajeAciertos|PorcentajeOmisiones|LatenciaDeRespuesta|TiempoEnRespuestaImpulsiva|Categorías|MaxAciertosSeguidos|TiempoTotalDelBloque|
|---|---|---|---|---|---|---|---|---|---|---|---|

In [33]:
registrosUnificados.loc[registrosUnificados['tipoPrueba'] == 'HAB'].head(1)

,IdAnimal,dia,mes,año,sexo,caja,marca,tipoPrueba,diaEnsayo,bloque,nombreArchivo,etiqueta
0,M-10-1,13,8,2021,M,10,1,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,


In [37]:
#Vamos a ver los datos que se tienen por cada animal
outputHabituacion=pd.DataFrame()
for animal in registrosUnificados['IdAnimal'].unique():
    output=pd.DataFrame()
    outputAnimal=pd.DataFrame()
    for indice, fila in registrosUnificados[ registrosUnificados['IdAnimal']==animal].iterrows():        
        if not (fila['etiqueta'] == '' or fila['etiqueta'] == 'TIEMPO'):
            continue
        else:
            try:
                # Datos que queremos guardar de cada registro
                tipoPrueba=fila['tipoPrueba']
                sexo=fila['sexo']
                
                # Convertimos el .docx a texto
                if tipoPrueba == 'HAB':
                    texto,esCSV=leerTexto(fila['nombreArchivo'])
                    registroSesion=extraerDatosBloque(texto,
                                                      esCSV,
                                                      fila['nombreArchivo'],
                                                      fila['diaEnsayo'],
                                                      fila['bloque'],
                                                      fila['IdAnimal'],
                                                      tipoPrueba,
                                                      fila['etiqueta']
                                                      )
                    output=pd.concat([output, registroSesion],  ignore_index=True)
            except:
                print(f"{fila['nombreArchivo']} Error al leer el archivo")
        
    output.to_excel(f'{animal}.xlsx')

El bloque D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE//MACHOS/AGOSTO/13.08.2021/13.08.2021. C10_1 HAB D1 B1.docx tiene 38 ensayos, pero termina en 50.0
El bloque D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE//MACHOS/AGOSTO/14.08.2021/14.08.2021. C10_1 HAB D2 B1.docx tiene 38 ensayos, pero termina en 50.0
El bloque D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE//MACHOS/AGOSTO/17.08.2021/17.08.2021. C11_1 HAB D3 B2.docx tiene 49 ensayos, pero termina en 50.0
El bloque D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE//MACHOS/AGOSTO/17.08.2021/17.08.2021. C11_3 HAB D3 B1.docx tiene 49 ensayos, pero termina en 50.0
El bloque D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE//MACHOS/AGOSTO/16.08.2021/16.08.2021. C11_4 HAB D2 B1.docx tiene 48 ensayos, pero termina en 50.0
D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE//MACHOS/AGOSTO/17.08.2021/17.08.20